In [86]:
from typing_extensions import TypedDict, Annotated
from langgraph.types import Send
from langgraph.graph import StateGraph, START, END
from langchain.chat_models import init_chat_model
from operator import add

llm = init_chat_model("openai:gpt-4o")

In [87]:
class State(TypedDict):

    document: str
    final_summary: str
    summaries: Annotated[list[dict], add]

In [88]:
def summarize_p(args):
    paragraph = args["paragraph"]
    index = args["index"]
    response = llm.invoke(f"Write a 3-sentence summary of this paragraph {paragraph}")
    return {"summaries": [{"summary": response.content, "index": index}]}


def dispatch_summarizers(state: State):
    chunks = state["document"].split("\n\n")
    return [
        Send("summarize_p", {"paragraph": chunk, "index": index})
        for index, chunk in enumerate(chunks)
    ]


def final_summary(state: State):
    response = llm.invoke(
        f"Using the following summaries, give me a final one {state['summaries']}"
    )
    return {"final_summary": response.content}

In [89]:
graph_builder = StateGraph(State)

graph_builder.add_node("summarize_p", summarize_p)
graph_builder.add_node("final_summary", final_summary)

graph_builder.add_conditional_edges(START, dispatch_summarizers, ["summarize_p"])
graph_builder.add_edge("summarize_p", "final_summary")
graph_builder.add_edge("final_summary", END)

graph = graph_builder.compile()

In [90]:
with open("fed_transcript.md", "r", encoding="utf-8") as file:
    document = file.read()

for chunk in graph.stream({"document": document}, stream_mode="updates"):
    print(chunk, "\n")

{'summarize_p': {'summaries': [{'summary': 'Payroll job gains have decreased significantly, averaging only 29,000 per month over the last three months. This slowdown is largely attributed to a reduction in labor force growth. The decline is driven by factors such as lower immigration and decreased labor force participation.', 'index': 6}]}} 

{'summarize_p': {'summaries': [{'summary': 'The unemployment rate increased slightly to 4.3 percent in August. Despite this rise, the rate has remained relatively stable over the past year. The current level is still considered low for the labor market.', 'index': 5}]}} 

{'summarize_p': {'summaries': [{'summary': 'Wage growth has been slowing down, yet it still surpasses the rate of inflation. This indicates that even with moderated growth, wages are increasing at a rate higher than inflation. Consequently, workers are experiencing improved purchasing power.', 'index': 8}]}} 

{'summarize_p': {'summaries': [{'summary': 'Long-term inflation expect